# Helper File

Testing the accuracy of the 4 algorithms in predicting characters for subtitle lines in 4 random episodes (that I have hand-labeled answers for)

In [ ]:
import sqlite3
import os
import re
import numpy as np
import pandas as pd

In [ ]:
# Testing a couple episodes (every line)
# 0201 is 89% guesses, 0908 is 99% guesses, 0512 is 95%
episodestotest = ['0201', '0908', '0512', '0307']
listfordf = []

conn = sqlite3.connect('/Users/Jack/Developer/friends/friendsdb.sqlite')
cur = conn.cursor()

for seasep in episodestotest:

    
    season = int(seasep[0:2])
    episode = int(seasep[2:4])
    
    cur.execute('''SELECT linenum, x FROM subs WHERE season=? AND episode=?''', (season, episode))
    
    for row in cur:
        linenum = str(row[0])
        x = row[1]
        splitlinenum = re.findall('(\d+|\D+)',linenum)
        num = int(splitlinenum[0])
        letter = ""
        if len(splitlinenum) > 1:
            letter = splitlinenum[1]
        listfordf.append([seasep, num, letter, x])
listfordf = sorted(listfordf)
for i, listitem in enumerate(listfordf):
    seasep = listitem[0]
    num = listitem[1]
    letter = listitem[2]
    x = listitem[3]
    linenum = str(num) + str(letter)
    listfordf[i] = [seasep, linenum, x]
# print(listfordf)
df = pd.DataFrame(listfordf, columns =['Episode', 'Linenum', 'X'])
# print(df)
df.to_csv('testepisodes2.csv', index=False)



In [ ]:
# Getting missing lines from all seasons, all episodes
def converttime(timestring):
    ms = int(timestring[9:])
    secs = int(timestring[6:8])
    mins = int(timestring[3:5])
    # not including ms here in order to "round down" to nearest second
    total = int((mins*60) + secs)
    return total


charstobefound = []

conn = sqlite3.connect('/Users/Jack/Developer/friends/friendsdb.sqlite')
cur = conn.cursor()

cur.execute('''SELECT season, episode, linenum, line, starttime FROM subs WHERE x='' ''')

for row in cur:
        season = row[0]
        episode = row[1]
        linenum = str(row[2])
        line = row[3]
        time = row[4]
#         print(season, episode, linenum, line)
        splitlinenum = re.findall('(\d+|\D+)',linenum)
        num = int(splitlinenum[0])
        letter = ""
        if len(splitlinenum) > 1:
            letter = splitlinenum[1]
        charstobefound.append([season, episode, num, letter, line, time])
charstobefound = sorted(charstobefound)
for i, listitem in enumerate(charstobefound):
    season = listitem[0]
    episode = listitem[1]
    num = listitem[2]
    letter = listitem[3]
    line = listitem[4]
    time = converttime(listitem[5])
    linenum = str(num) + str(letter)
    charstobefound[i] = [season, episode, linenum, line, time]
print(charstobefound)
df = pd.DataFrame(charstobefound, columns =['Season', 'Episode', 'Linenum', 'Line', 'Time'])
# print(df)
df.to_csv('charstobefound.csv', index=False)
